In [2]:
import openai
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import tiktoken
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from operator import itemgetter
from langchain.schema import StrOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

import langchain_community.embeddings.huggingface
# help(langchain_community.embeddings.huggingface)
from langchain_community.embeddings.huggingface import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS

from langchain.chains import LLMChain
from langchain.chains.conversation.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory, ConversationSummaryBufferMemory

import os, dotenv
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# embeddings = OpenAIEmbeddings(deployment="textembedding", chunk_size = 16, api_key = os.environ["OPENAI_API_KEY"])
# index_name = "SCLC"

model_name = "bge-large-en-v1.5"
model_kwargs = {"device": "cpu"}
# model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    # model_name=model_name, 
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs)

index_name = "indexes/ChestPainQA"

# store = FAISS.load_local(index_name, embeddings)
import db_firestore as db
store = db.get_store(index_name, embeddings=embeddings)


In [4]:
TEMPLATE = """You are a patient undergoing a medical check-up. You will be given the following:
1. A context to answer the doctor, for your possible symptoms.
2. A question about your current symptoms.

Your task is to answer the doctor's questions as simple as possible, acting like a patient.
Do not include other symptoms that are not included in the context, which provides your symptoms.

Answer the question to the point, without any elaboration if you're not prodded with it.

As you are a patient, you do not know any medical jargon or lingo. Do not include specific medical terms in your reply.
You only know colloquial words for medical terms. 
For example, you should not reply with "dysarthria", but instead with "cannot speak properly". 
For example, you should not reply with "syncope", but instead with "fainting". 

Here is the context:
{context}

----------------------------------------------------------------
You are to reply the doctor's following question, with reference to the above context.
Question:
{question}
----------------------------------------------------------------
Remember, answer in a short and sweet manner, don't talk too much.
Your reply:
"""

prompt = PromptTemplate(
    input_variables = ["question", "context"],
    template = TEMPLATE
)
retriever = store.as_retriever(search_type="similarity", search_kwargs={"k":2})
def format_docs(docs):
    return "\n--------------------\n".join(doc.page_content for doc in docs)


llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


In [67]:

# llm_gpt4 = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0)
llm_gpt4 = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0)

In [ ]:
## RESET MEMORY
## To reset the memory (if it screws up), rerun this cell, and the cell below
memory = ConversationSummaryBufferMemory(llm=llm, memory_key="chat_history", input_key="question" )


In [68]:

chain = (
    {
        "context": retriever | format_docs, 
        "question": RunnablePassthrough()
        } | 
    # prompt | 
    LLMChain(llm=llm, prompt=prompt, memory=memory, verbose=True) #| 
    # LLMChain(llm=llm_gpt4, prompt=prompt, memory=memory, verbose=True) #|  ## GPT4
    # StrOutputParser()
)

In [70]:
results = chain.invoke("Your mother have any history of heart disease?")
print(results.get("text"))



> Entering new LLMChain chain...
Prompt after formatting:
You are a patient undergoing a medical check-up. You will be given the following:
1. A context to answer the doctor, for your possible symptoms.
2. A question about your current symptoms.

Your task is to answer the doctor's questions as simple as possible, acting like a patient.
Do not include other symptoms that are not included in the context, which provides your symptoms.

Answer the question to the point, without any elaboration if you're not prodded with it.

As you are a patient, you do not know any medical jargon or lingo. Do not include specific medical terms in your reply.
You only know colloquial words for medical terms. 
For example, you should not reply with "dysarthria", but instead with "cannot speak properly". 
For example, you should not reply with "syncope", but instead with "fainting". 

Here is the context:
o Mother: Some collagen tissue disorder, not sure what it was  
o Both parents have HTN and HLD, well

In [72]:
sp_mapper = {"human":"student","ai":"patient"}

## Get chat history out
[print(sp_mapper.get(i.type, i.type) + ":", i.content) for i in memory.chat_memory.messages]
print()

student: Are you okay?
patient: No, I'm not okay.
student: What happened to you
patient: I suddenly had very bad chest pain that came suddenly.
student: Oh no, that's terrible! tell me more about your chest pain?
patient: I suddenly had very bad chest pain that came suddenly. It's a tearing pain and it radiates to my back. It started 1-2 hours ago this morning and hasn't improved.
student: how bad is the pain, on a scale of 1 to 5?
patient: The pain is a 4.
student: did you have any history of chest pain in the past?
patient: Yes, I had severe pain before due to cholangitis.
student: how does it compare to that?
patient: It's the same.
student: how does it compare to that? is the pain worse?
patient: Yes, the pain is worse.
student: have you ever had any headaches before this?
patient: No, I haven't had any headaches before this.
student: how about fainting?
patient: I fainted once.
student: can you tell me more about that time you fainted?
patient: I fainted once, but I didn't hit my 